<a href="https://colab.research.google.com/github/gkuch22/ml-final/blob/main/final_model_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch # Main PyTorch Library
import matplotlib.pyplot as plt # Used for visualizing the images and plotting the training progress
import pandas as pd # Used to read/create dataframes (csv) and process tabular data
import numpy as np # preprocessing and numerical/mathematical operations
import seaborn as sns

device = "cuda" if torch.cuda.is_available() else "cpu" # detect the GPU if any, if not use CPU, change cuda to mps if you have a mac
print("Device available: ", device)


Device available:  cuda


In [10]:
train_df = pd.read_csv('/content/drive/MyDrive/cs231n/assignments/final/datasets/train.csv')
stores_df = pd.read_csv('/content/drive/MyDrive/cs231n/assignments/final/datasets/stores.csv')
features_df = pd.read_csv('/content/drive/MyDrive/cs231n/assignments/final/datasets/features.csv')
final_df = pd.read_csv('/content/drive/MyDrive/cs231n/assignments/final/datasets/test.csv')

In [11]:
train_df

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [18]:
!pip install neuralforecast -q

In [19]:
!pip install dagshub mlflow -q

In [5]:
import dagshub
import mlflow

dagshub.init(repo_owner='gkuch22', repo_name='ml-final', mlflow=True)

Accessing as gkuch22

Initialized MLflow to track repo "gkuch22/ml-final"

Repository gkuch22/ml-final initialized!

In [20]:
import mlflow

import os
os.environ["MLFLOW_TRACKING_USERNAME"] = "gkuch22"
os.environ["MLFLOW_TRACKING_PASSWORD"] = ""

mlflow.set_tracking_uri("https://dagshub.com/gkuch22/ml-final.mlflow")

logged_model = "models:/best_model_patchtst/1"

loaded_model = mlflow.sklearn.load_model(logged_model)

In [17]:
predictions = loaded_model.predict(final_df)

In [13]:
predictions

,unique_id,ds,y,PatchTST
0,10_1,2012-11-02,NaN,67925.046875
1,10_1,2012-11-09,NaN,44971.558594
2,10_1,2012-11-16,NaN,39217.718750
3,10_1,2012-11-23,NaN,44840.160156
4,10_1,2012-11-30,NaN,50270.113281
...,...,...,...,...
115059,9_98,2013-01-04,NaN,9.649915
115060,9_99,2012-12-07,NaN,NaN
115061,9_99,2013-07-05,NaN,NaN
115062,9_99,2013-07-19,NaN,NaN


In [14]:
predictions = predictions.drop(columns=['y'])
predictions['PatchTST'] = predictions['PatchTST'].fillna(0)
submission = predictions.copy()

submission['Id'] = submission['unique_id'] + '_' + submission['ds'].astype(str)
submission = submission.rename(columns={'PatchTST': 'Weekly_Sales'})
submission = submission[['Id', 'Weekly_Sales']].reset_index(drop=True)

In [15]:
submission

,Id,Weekly_Sales
0,10_1_2012-11-02,67925.046875
1,10_1_2012-11-09,44971.558594
2,10_1_2012-11-16,39217.718750
3,10_1_2012-11-23,44840.160156
4,10_1_2012-11-30,50270.113281
...,...,...
115059,9_98_2013-01-04,9.649915
115060,9_99_2012-12-07,0.000000
115061,9_99_2013-07-05,0.000000
115062,9_99_2013-07-19,0.000000


In [16]:
submission.to_csv('patchtst_submission.csv', index=False)